# Wep Scraping 

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import time

driver = webdriver.Chrome()
driver.get('https://www.saudiexchange.sa/wps/portal/saudiexchange/ourmarkets/main-market-watch/main-market-performance?locale=ar')

try:
    dropdown = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "sectorList"))
    )
except Exception as e:
    print("Error: ", e)
    driver.quit()

all_data = pd.DataFrame()

select = Select(dropdown)
for option in select.options:
    if option.text == "كل السوق":
        continue
    
    select.select_by_visible_text(option.text)
    
    time.sleep(5)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    table_div = soup.find('div', {'class': 'dataTables_scrollBody'})
    if table_div:
        table = table_div.find('table')
    
    
    headers = []
    if table:
        header_row = table.find('thead').find_all('th')
        headers = [th.text.strip() for th in header_row]
    
    rows = []
    if table:
        tbody = table.find('tbody')
        if tbody:
            row_tags = tbody.find_all('tr')
            for tr in row_tags:
                cells = tr.find_all('td')
                row = [cell.text.strip() for cell in cells]
                if row:
                    rows.append(row)
    
    
    if headers and rows and len(headers) == len(rows[0]):
        sector_df = pd.DataFrame(rows, columns=headers)
    else:
        
        max_columns = max(len(row) for row in rows)
        if len(headers) != max_columns:
            headers = [f"Column {i+1}" for i in range(max_columns)]
        sector_df = pd.DataFrame(rows, columns=headers)
    
    sector_df['Sector'] = option.text
    
    all_data = pd.concat([all_data, sector_df], ignore_index=True)

driver.quit()

print(all_data)

              الشركة الإفتتاح  الأعلى الأدنى الإغلاق التغير % التغيير  \
0    أرامكو السعودية    34.85   35.15   26.8   27.95   -6.9    -19.80   
1               أديس    17.54    26.1  14.84   20.48   2.94     16.76   
2          بترو رابغ    10.56   11.06   6.58     8.2  -2.36    -22.35   
3             البحري    23.52    30.4  21.36    29.6   6.08     25.85   
4             الدريس   102.49  147.81  82.53   123.2  20.71     20.21   
..               ...      ...     ...    ...     ...    ...       ...   
204              سمو    39.77   51.86   27.5    42.6   2.83      7.12   
205             طيبة    28.40    48.9   25.1    44.6   16.2     57.04   
206     البحر الأحمر    24.08    49.6   19.9    46.7  22.62     93.94   
207              مكة    72.00   119.8     61   113.8   41.8     58.06   
208          الأندلس    24.78   30.95  18.82    24.3  -0.48     -1.94   

    الكمية المتداولة القيمة المتداولة (ر.س)                 Sector  
0      5,543,658,787       2,085,715,602.30           

In [11]:
all_data

,الشركة,الإفتتاح,الأعلى,الأدنى,الإغلاق,التغير,% التغيير,الكمية المتداولة,القيمة المتداولة (ر.س),Sector
0,أرامكو السعودية,34.85,35.15,26.8,27.95,-6.9,-19.80,"5,543,658,787","2,085,715,602.30",الطاقة
1,أديس,17.54,26.1,14.84,20.48,2.94,16.76,"1,560,799,272","66,412,733.50",الطاقة
2,بترو رابغ,10.56,11.06,6.58,8.2,-2.36,-22.35,"629,672,963","108,450,108.56",الطاقة
3,البحري,23.52,30.4,21.36,29.6,6.08,25.85,"163,385,654","14,383,934.25",الطاقة
4,الدريس,102.49,147.81,82.53,123.2,20.71,20.21,"96,635,167","42,924,622.80",الطاقة
...,...,...,...,...,...,...,...,...,...,...
204,سمو,39.77,51.86,27.5,42.6,2.83,7.12,"88,393,382","6,558,809.05",إدارة وتطوير العقارات
205,طيبة,28.40,48.9,25.1,44.6,16.2,57.04,"80,681,597","4,600,490.60",إدارة وتطوير العقارات
206,البحر الأحمر,24.08,49.6,19.9,46.7,22.62,93.94,"59,466,985","166,786,815.50",إدارة وتطوير العقارات
207,مكة,72.00,119.8,61,113.8,41.8,58.06,"40,023,142","6,570,656.00",إدارة وتطوير العقارات


In [13]:
all_data.to_csv('saudi_exchange_sector_data.csv', index=False)